In [ ]:
import os
import torch
from sentence_transformers import SentenceTransformer
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core.prompts.prompts import SimpleInputPrompt
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.core import ServiceContext
from llama_index.legacy.embeddings.langchain import LangchainEmbedding

In [ ]:
system_prompt="""
You are a Q&A assistant. Your goal is to answer questions as accurately as possible based on the instructions and context provided.
"""
#default format supportable by LLama2
query_wrapper_prompt=SimpleInputPrompt("<|USER|> {query_str}<|ASSISTANT|>")

In [ ]:
llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature":0.0, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
    model_name="meta-llama/Llama-2-7b-chat-hf",
    device_map="auto",
    model_kwargs={"torch_dtype": torch.float16, "load_in_8bit":True}

)
model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")
embed_model = LangchainEmbedding(
    HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"))



In [ ]:
service_context = ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embed_model
)

In [ ]:
# Now, to load the persisted index and use it:
from llama_index.core import StorageContext, load_index_from_storage

# Load the storage context
storage_context = StorageContext.from_defaults(persist_dir="/content/storage")

# Load the index from the storage
loaded_index = load_index_from_storage(storage_context, service_context=service_context)

# Use the loaded query engine to answer your query
response = loaded_query_engine.query("What are the OTC medication for alopecia areata?")
print(response)